In [1]:
from torchtext import data
from torchtext.vocab import Vectors
from torch.nn import init
from tqdm import tqdm
import numpy as np
import os
from random import shuffle
import pandas as pd
import torch
import time
import random

In [2]:
Text = []
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()
with open('/Users/denhiroshi/Downloads/Embeding/sentence.txt') as f:
    Text = f.readlines()
Text = Text[1:]
len(Text),len(Y)

(467, 467)

In [3]:
write_csv = []
for i in range(len(Text)):
    write_csv.append((str(i) , Text[i][:-1], int(Y[i])))
shuffle(write_csv)
len(write_csv),write_csv[0]
train_csv_data = write_csv[0 : int(0.7*len(write_csv))]
test_csv_data = write_csv[int(0.7*len(write_csv)) : ]

In [4]:
import csv
headers = ['id', 'Text', 'label']
with open('train_glove.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(train_csv_data)
    
with open('test_glove.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(test_csv_data)

In [5]:
def get_dataset(csv_data, text_field, label_field, test=False):

    fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("Text", text_field), ("label", label_field)]       
    examples = []

    if test:
        # 如果为测试集，则不加载label
        for text in tqdm(csv_data['Text']):
            examples.append(data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in tqdm(zip(csv_data['Text'], csv_data['label'])):
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields
tokenize = lambda x: x.split()
train_data = pd.read_csv('train_glove.csv')
test_data = pd.read_csv("test_glove.csv")
print(len(train_data))
print(len(test_data))

TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = data.Field(sequential=False, use_vocab=False)

# 得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
test_examples, test_fields = get_dataset(test_data, TEXT, LABEL)
# 构建Dataset数据集
train = data.Dataset(train_examples, train_fields)
test = data.Dataset(test_examples, test_fields)
train,test

326it [00:00, 73822.65it/s]
141it [00:00, 52554.60it/s]

326
141


(<torchtext.data.dataset.Dataset at 0x12d362f28>,
 <torchtext.data.dataset.Dataset at 0x12d4d7518>)

In [6]:
train.examples[0].Text,test.examples[0].Text

(['laser',
  'profiling',
  'is',
  'a',
  'technique',
  'for',
  'the',
  'study',
  'of',
  'prosthetic',
  'heart',
  'valve',
  'leaflet',
  'motion.'],
 ['the',
  'ancients',
  'cultivated',
  'the',
  'species',
  "'heracleum",
  "sphondylium'",
  'which',
  'is',
  'the',
  'parsnip',
  'a',
  'similar',
  'plant',
  'to',
  'carrots.'])

In [7]:
TEXT.build_vocab(train)
TEXT.build_vocab(test)
vocab = TEXT.vocab
vocab.freqs.most_common(10)

[('the', 235),
 ('a', 92),
 ('of', 83),
 ('and', 80),
 ('to', 53),
 ('is', 46),
 ('in', 42),
 ('from', 36),
 ('for', 34),
 ('that', 28)]

In [8]:
cache = '.vector_cache'
if not os.path.exists(cache):
    os.mkdir(cache)
vectors = Vectors(name='/Users/denhiroshi/Downloads/glove/glove.6B.200d.txt', cache=cache)
TEXT.build_vocab(train, vectors=vectors)
TEXT.build_vocab(test, vectors=vectors)

In [9]:
torch.sum(TEXT.vocab.vectors != 0)/200,TEXT.vocab.vectors.shape

(tensor(1014), torch.Size([1310, 200]))

In [10]:
from torchtext.data import Iterator, BucketIterator
train_iter = Iterator(dataset = train, batch_size=64, device=-1, shuffle=True, sort_within_batch=False, repeat=False)
test_iter = Iterator(dataset = test, batch_size=64, device=-1, sort=False, sort_within_batch=False, repeat=False)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [11]:
weight_matrix = TEXT.vocab.vectors

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
weight_matrix = TEXT.vocab.vectors

In [13]:
class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()
        self.word_embeddings = nn.Embedding(len(TEXT.vocab), 200)  # embedding之后的shape: torch.Size([200, 8, 300])
        self.word_embeddings.weight.data.copy_(weight_matrix)
        self.lstm = nn.LSTM(input_size=200, hidden_size=128, num_layers=1)  # torch.Size([200, 8, 128])
        self.decoder = nn.Linear(128, 7)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out,(h_n,c_n) = self.lstm(embeds)
        final = h_n[-1]
        y = self.decoder(final)
        return y

In [14]:
model = LSTM()
model.train()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
loss_funtion = F.cross_entropy
for epoch in range(1,300):
    for step, batch_train in enumerate(train_iter):
        optimizer.zero_grad()
        predicted = model(batch_train.Text)
        loss = loss_funtion(predicted, batch_train.label)
        loss.backward()
        optimizer.step()
        acc = 0
        counter = 0
        if epoch % 10 == 0:
            acc = 0
            counter = 0
            with torch.no_grad():
                for counts, batch_test in enumerate(test_iter):
                    test_pred = model(batch_test.Text)
                    prob = torch.nn.functional.softmax(test_pred,dim=1)
                    pred_cls = torch.argmax(prob,dim=1)
                    acc += (pred_cls.int() == batch_test.label.int()).sum().numpy() / pred_cls.size()[0]
                    counter += 1
            acc /= counter
            print(f"epoch:{epoch} loss:{loss} acc:{acc}")

epoch:10 loss:1.9376752376556396 acc:0.1141826923076923
epoch:10 loss:1.925981879234314 acc:0.1346153846153846
epoch:10 loss:1.937678575515747 acc:0.15504807692307693
epoch:10 loss:1.9398982524871826 acc:0.09375
epoch:10 loss:1.9361587762832642 acc:0.15504807692307693
epoch:10 loss:1.788271427154541 acc:0.1141826923076923


KeyboardInterrupt: 